<a href="https://colab.research.google.com/github/subikkshas/DA6401/blob/main/DLass1Q4to6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import fashion_mnist
import numpy as np
from  matplotlib import pyplot as plt
import time
import math
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

In [ ]:
import wandb
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: subikksha (subikksha-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
dataset= fashion_mnist.load_data()
(X_train_and_validation, y_train_and_validation), (X_test, y_test) = dataset
X_train, X_validation, y_train, y_validation = train_test_split(X_train_and_validation, y_train_and_validation, test_size=0.1, random_state=42)
X_train = (X_train/255.0).astype(np.float32)
X_validation = (X_validation/255.0).astype(np.float32)
X_test = (X_test/255.0).astype(np.float32)

print("Train Dataset Shape: ", X_train.shape)
print("Train Target Vector Shape: ", y_train.shape)
print("Test Dataset Shape:", X_test.shape)
print("Test Target Vector Shape", y_test.shape)
print("Validation Dataset Shape:", X_validation.shape)
print("Validation Target Vector Shape", y_validation.shape)




29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train Dataset Shape:  (54000, 28, 28)
Train Target Vector Shape:  (54000,)
Test Dataset Shape: (10000, 28, 28)
Test Target Vector Shape (10000,)
Validation Dataset Shape: (6000, 28, 28)
Validation Target Vector Shape (6000,)


In [ ]:
X_train = np.array(X_train.reshape(X_train.shape[0], 784,1))
X_test = np.array(X_test.reshape(X_test.shape[0], 784,1))
X_validation = np.array(X_validation.reshape(X_validation.shape[0], 784,1))

In [ ]:
def layer_init(arr,n1,n2,init_type):
    np.random.seed(10)
    if init_type=="random":
        arr.append(np.random.randn(n1,n2)*0.1)
    elif init_type=="xavier":
        arr.append(np.random.randn(n1,n2)*np.sqrt(2/(n1+n2)))
    return arr

def param(layers,init_type):
    W=[]
    B=[]
    for i in range(len(layers)-1):
        W=layer_init(W,layers[i+1],layers[i],init_type)
        B=layer_init(B,layers[i+1],1,init_type)
    return W,B

#Activation function
def activation(activation_function):
    if activation_function == 'sigmoid':
        return sigmoid
    if activation_function == 'tanh':
        return tanh
    if activation_function == 'ReLU':
        return relu

def sigmoid(x, derivative = False):
    if derivative:
        return sigmoid(x)*(1-sigmoid(x))
    return 1/(1 + np.exp(-x))

def tanh(x, derivative = False):
    if derivative:
        return 1 - tanh(x)**2
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

def relu(x, derivative = False):
    if derivative:
        return (x>0)*1
    return x*(x>0)

def softmax(x,derivative = False):
    if derivative:
        return softmax(x)*(1- softmax(x))
    return np.exp(x)/np.sum(np.exp(x), axis = 0)

def one_hot(y, num_output_nodes):
    v = np.zeros((num_output_nodes, len(y)))
    for i,j in enumerate(y):
        v[j,i] = 1
    return v


def forward(x, W, B, activation_type):
    h = []
    a = []
    sigma = activation(activation_type)  #activation
    h.append(x)   #h0 = x
    a.append(np.dot(W[0], h[0]) + B[0])
    for i in range(len(W)-1):
        h.append(sigma(a[-1]))
        a.append(np.dot(W[i+1], h[-1]) + B[i+1])
    y_hat = softmax(a[-1])

    return y_hat, h, a



def loss(y,y_hat,l_type,W,reg,n_class):
    if l_type=='cross_entropy':
        err=-1*np.sum(np.multiply(one_hot(y,n_class),np.log(y_hat)))/one_hot(y,n_class).shape[1]
    elif l_type=='squared_error':
        err=np.sum((one_hot(y,n_class)-y_hat)**2)/(2*one_hot(y,n_class)).shape[1]

    if W:
        r=0
        for i in range(len(W)):
            r+=np.sum((np.array(W,dtype=object)**2)[i])
        err=err+reg*r
    return err

def eval_acc(y_hat, y_true):
    return np.mean(np.argmax(y_hat, axis = 0) ==y_true )*100


In [ ]:
def back_prop(x, y, y_hat, a, h , W, B, batch_size,l_type,act_type):
    grad_h,grad_a,grad_W,grad_B = [0]*len(h),[0]*len(a),[0]*len(W),[0]*len(B)
    sigma = activation(act_type)

    if l_type == "cross_entropy":
        grad_h[-1] = -1*(y/y_hat)
        grad_a[-1] = -1*(y-y_hat)
    if l_type == "squared_error":   ##### edit this
        grad_h[-1] = y_hat - y
        grad_a[-1] = (y_hat - y)*softmax(a[-1])*(1-softmax(a[-1]))

    for i in range(len(W)-1, -1, -1):
        grad_W[i] = np.dot(grad_a[i], h[i].T)
        grad_B[i] = np.dot(grad_a[i], np.ones((grad_a[i].shape[1], 1)))
        if i > 0:
            grad_h[i-1] = np.dot(W[i].T, grad_a[i])
            grad_a[i-1]  = np.multiply(grad_h[i-1],sigma(a[i-1], derivative = True))

    return grad_W, grad_B, grad_h, grad_a

In [ ]:
def sgd_step(W,B,grad_W,grad_B,lr,reg):
    W=np.array(W,dtype=object)
    B=np.array(B,dtype=object)
    W-=lr*reg*W+lr*np.array(grad_W,dtype=object)
    B-=lr*reg*B+lr*np.array(grad_B,dtype=object)

    return W.tolist(),B.tolist()



def momentum_step(w, b, gW, gB, lr=0.001, gamma=0.9, reg=0):
    params = {'w': w, 'b': b}

    Wmoments = [np.zeros_like(p) for p in params['w']]
    Bmoments = [np.zeros_like(p) for p in params['b']]

    Wmoments = gamma * np.array(Wmoments,dtype=object) + lr * np.array(gW,dtype=object)
    W = (1 - lr * reg) * np.array(params['w'],dtype=object) - Wmoments
    Wmoments = Wmoments.tolist()

    Bmoments = gamma * np.array(Bmoments,dtype=object) + lr * np.array(gB,dtype=object)
    B = (1 - lr * reg) * np.array(params['b'],dtype=object) - Bmoments
    Bmoments = Bmoments.tolist()

    return W.tolist(), B.tolist()


def RMSprop_step(w, b, gW, gB, lr=0.01, beta=0.99):
    params = {'w': w, 'b': b}

    vW = [np.zeros_like(p) for p in params['w']]
    vB = [np.zeros_like(p) for p in params['b']]

    vW = beta * np.array(vW, dtype=object) + (1 - beta) * (np.array(gW, dtype=object) ** 2)
    W = np.array(params['w'], dtype=object) - (lr / ((vW + 1e-7) ** 0.5)) * np.array(gW, dtype=object)

    vB = beta * np.array(vB, dtype=object) + (1 - beta) * (np.array(gB, dtype=object) ** 2)
    B = np.array(params['b'], dtype=object) - (lr / ((vB + 1e-7) ** 0.5)) * np.array(gB, dtype=object)

    return W.tolist(), B.tolist()



In [ ]:
import wandb


def train(X_train, y_train, x_val, y_val, num_inputs_nodes, hidden_layers, out_num, init_type, epochs,
          batch_size, l_type, act_type, op_name, lr_rate, reg, run_name):

    # Ensure WandB starts fresh
    wandb.finish()

    run = wandb.init(project="DA6401-Assignment-1", nam e=run_name, reinit=True)

    in_node = [num_inputs_nodes]
    layers = []
    layers.extend(in_node)
    layers.extend(hidden_layers)
    layers.extend([out_num])

    W, B = param(layers, init_type)
    N = X_train.shape[0]
    n_batches = int(np.floor(N / batch_size))

    for epoch in range(epochs):
        train_loss, train_accuracy, val_loss, val_accuracy = [], [], [], []
        l, acc, ds, steps = 0, 0, 0, 1

        while ds < N:
            mini_batch_size = min((N - ds), batch_size)
            x = np.squeeze(X_train[ds:ds + mini_batch_size]).T
            y = one_hot(y_train[ds:ds + mini_batch_size], out_num)

            y_hat, h, a = forward(x, W, B, act_type)
            grad_W, grad_B, grad_h, grad_a = back_prop(x, y, y_hat, a, h, W, B, batch_size, l_type, act_type)

            if op_name == 'sgd':
                W, B = sgd_step(W, B, grad_W, grad_B, lr_rate, reg)
            elif op_name == 'momentum':
                W, B = momentum_step(W, B, grad_W, grad_B, lr_rate, reg)
            elif op_name == 'rmsprop':
                W, B = RMSprop_step(W, B, grad_W, grad_B, lr_rate, reg)

            l += loss(y_train[ds:ds + mini_batch_size], y_hat, l_type, W, reg, out_num)
            acc += eval_acc(y_hat, y_train[ds:ds + mini_batch_size])

            steps += 1
            ds += batch_size

        l /= (n_batches + mini_batch_size)
        acc /= steps

        train_loss.append(l)
        train_accuracy.append(acc)

        y_val_hat, _, _ = forward(np.squeeze(x_val).T, W, B, act_type)
        val_acc = eval_acc(y_val_hat, y_val)
        val_l = loss(y_val, y_val_hat, l_type, W=None, reg=reg, n_class=out_num)

        val_accuracy.append(val_acc)
        val_loss.append(val_l)

        wandb.log({"epoch": epoch, "Train_loss": l, "Train_acc": acc, "val_loss": val_l, "val_Accuracy": val_acc})

        #print(f"Epoch {epoch}: Train Loss = {l:.4f}, Train Accuracy = {acc:.4f}, Val Loss = {val_l:.4f}, Val Accuracy = {val_acc:.4f}")

    wandb.finish()  # Explicitly finish the WandB run

    return W, B, train_loss, train_accuracy, val_loss, val_accuracy


In [ ]:
hidden_layers=[64,32,16]
init_type='random'
epochs=10
batch_size=50
loss_type='squared_error'
optimizer_name='rmsprop'
learning_rate=1e-3
reg_lamda=0
activation_type='sigmoid'

In [ ]:
wandb.init(project="DA6401-Assignment-1")
fit=train(X_train, y_train, X_validation, y_validation,784,hidden_layers, 10, init_type, epochs, batch_size,
                                                                 loss_type, activation_type, optimizer_name, learning_rate,
                                                                 reg_lamda)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: subikksha (subikksha-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Train_acc,▁▅▆▆▇▇▇███
Train_loss,█▅▃▃▂▂▂▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
val_Accuracy,▁▄▅▆▇▇▇███
val_loss,█▅▄▃▂▂▂▁▁▁
Train_acc,84.07956
Train_loss,0.2313
epoch,9
val_Accuracy,83.65
val_loss,0.24556


In [ ]:
import wandb

In [ ]:
sweep_configuration = {'method'    : "random",  #Other available--> grid, bayes
                       'metric'    : {'name': 'val_Accuracy','goal':'maximize'},
                       'parameters': {'epochs':{'values':[ 5, 10]},
                                      'hidden_layers':{'values':[[64,32],[64,32,16],[128,64,32],[128,64,32,16],[128,32,32,16]] }, #change needed
                                      'learning_rate':{'values':[1e-3,1e-4]},
                                      'weight_decay':{'values':[0, 0.0005, 0.5]},
                                      'optimizer_name': {'values':['sgd', 'momentum', 'rmsprop', 'nesterov','adam']},
                                      'batch_size':{'values':[16,32,64]},
                                      'init_type': {'values':['random','xavier']},
                                      'activation_type':{'values':['sigmoid','tanh','ReLU']},
                                      'loss_type': {'values':['cross_entropy']} }}

In [ ]:
def sweep_train():
    hyperparameters = dict(epochs=5,
                           hidden_layers=[64, 32],
                           learning_rate=1e-4,
                           weight_decay=0,
                           optimizer_name='sgd',
                           batch_size=16,
                           init_type='random',
                           activation_type='sigmoid',
                           loss_type='cross_entropy',
                           reg_lamda=0)

    wandb.init(project="DA6401-Assignment-1", entity="subikksha", config=hyperparameters)
    config = wandb.config
    epochs = config.epochs
    hidden_layers = config.hidden_layers
    learning_rate = config.learning_rate
    weight_decay = config.weight_decay
    optimizer_name = config.optimizer_name
    batch_size = config.batch_size
    init_type = config.init_type
    activation_type = config.activation_type
    loss_type = config.loss_type
    reg_lamda = config.reg_lamda
    wandb.run.name = "e_{}_hl_{}_lr_{}_wd_{}_o_{}_bs_{}_winit_{}_ac_{}_los_{}_r_{}".format(epochs, \
                                                                                           hidden_layers, \
                                                                                           learning_rate, \
                                                                                           weight_decay, \
                                                                                           optimizer_name, \
                                                                                           batch_size, \
                                                                                           init_type, \
                                                                                           activation_type, \
                                                                                           loss_type, \
                                                                                           reg_lamda)

    _, _, train_loss, train_accuracy, val_loss, val_accuracy = train(X_train, y_train, X_validation, y_validation, 784,
                                                                     hidden_layers, 10, init_type, epochs, batch_size,
                                                                     loss_type, activation_type, optimizer_name,
                                                                     learning_rate, reg_lamda, run_name=f"run_{epochs}_{batch_size}_{optimizer_name}")

In [ ]:
import wandb

def sweep_train():
    wandb.finish()  # Ensure no previous session is hanging

    hyperparameters = dict(
        epochs=5,
        hidden_layers=[128, 64, 32],
        learning_rate=1e-3,
        weight_decay=0.0005,
        optimizer_name='nesterov',
        batch_size=32,
        init_type='xavier',
        activation_type='ReLU',
        loss_type='cross_entropy',
        reg_lamda=0
    )

    run = wandb.init(project="DA6401-Assignment-1", config=hyperparameters)

    config = wandb.config
    run_name = f"hl_{len(config.hidden_layers)}_bs_{config.batch_size}_ac_{config.activation_type}"
    wandb.run.name = run_name

    _, _, train_loss, train_accuracy, val_loss, val_accuracy = train(
        X_train, y_train, X_validation, y_validation, 784,
        config.hidden_layers, 10, config.init_type, config.epochs, config.batch_size,
        config.loss_type, config.activation_type, config.optimizer_name,
        config.learning_rate, config.reg_lamda, run_name=run_name
    )

    wandb.finish()


In [ ]:
sweep_id = wandb.sweep(sweep_configuration,project="DA6401-Assignment-1")
wandb.agent(sweep_id,function=sweep_train,project="DA6401-Assignment-1",count=100)

Create sweep with ID: nn3r7wsk
Sweep URL: https://wandb.ai/subikksha-indian-institute-of-technology-madras/DA6401-Assignment-1/sweeps/nn3r7wsk


wandb: Agent Starting Run: yjpv5g2g with config:
wandb: 	activation_type: sigmoid
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	hidden_layers: [128, 64, 32, 16]
wandb: 	init_type: random
wandb: 	learning_rate: 0.0001
wandb: 	loss_type: cross_entropy
wandb: 	optimizer_name: sgd
wandb: 	weight_decay: 0


Train_acc,▆▃▁▃▄▄▅▆▇█
Train_loss,█▁▁▁▁▁▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
val_Accuracy,▁█▄▃▃▃▃▃▃▄
val_loss,█▄▃▃▂▂▂▂▁▁
Train_acc,9.89275
Train_loss,2.1812
epoch,9
val_Accuracy,10.36667
val_loss,2.30247


wandb: Agent Starting Run: p88r2ayz with config:
wandb: 	activation_type: ReLU
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	hidden_layers: [64, 32]
wandb: 	init_type: xavier
wandb: 	learning_rate: 0.0001
wandb: 	loss_type: cross_entropy
wandb: 	optimizer_name: adam
wandb: 	weight_decay: 0.0005


Train_acc,▁▁▁▁▁
Train_loss,▁▁▁▁▁
epoch,▁▃▅▆█
val_Accuracy,▁▁▁▁▁
val_loss,▁▁▁▁▁
Train_acc,9.99384
Train_loss,2.52659
epoch,4
val_Accuracy,9.95
val_loss,2.66088


...

Train_acc,▁▁▁▁▁
Train_loss,▁▁▁▁▁
epoch,▁▃▅▆█
val_Accuracy,▁▁▁▁▁
val_loss,▁▁▁▁▁
Train_acc,9.99384
Train_loss,2.33225
epoch,4
val_Accuracy,9.95
val_loss,2.47428


wandb: Agent Starting Run: taw9wx9v with config:
wandb: 	activation_type: ReLU
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	hidden_layers: [64, 32, 16]
wandb: 	init_type: xavier
wandb: 	learning_rate: 0.001
wandb: 	loss_type: cross_entropy
wandb: 	optimizer_name: adam
wandb: 	weight_decay: 0.0005


Train_acc,▁▁▁▁▁
Train_loss,▁▁▁▁▁
epoch,▁▃▅▆█
val_Accuracy,▁▁▁▁▁
val_loss,▁▁▁▁▁
Train_acc,8.56139
Train_loss,2.55281
epoch,4
val_Accuracy,8.51667
val_loss,2.71155


wandb: Agent Starting Run: 1ow0dsr6 with config:
wandb: 	activation_type: ReLU
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_layers: [64, 32, 16]
wandb: 	init_type: xavier
wandb: 	learning_rate: 0.001
wandb: 	loss_type: cross_entropy
wandb: 	optimizer_name: nesterov
wandb: 	weight_decay: 0


Train_acc,▁▁▁▁▁▁▁▁▁▁
Train_loss,▁▁▁▁▁▁▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
val_Accuracy,▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▁▁▁▁▁▁▁
Train_acc,8.56968
Train_loss,2.68221
epoch,9
val_Accuracy,8.51667
val_loss,2.71155
